1. 給予文章內容
2. 對於每篇文章給予分類: A,B,C
3. 建立＆測試模型
4. 預測分類
5. 預測率太低的處理

In [19]:
import cPickle as pickle
import numpy as np
from sklearn.datasets.base import Bunch
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB #導入多項式貝葉斯算法
from sklearn import metrics
import jieba

# from __future__ import print_function, unicode_literals
# import os
filepath="../data/zh_dic.txt"
jieba.load_userdict(filepath)

# Bunch類提供一種key,value的對象形式
# target_name:所有分類集名稱列表
# label:每個文件的分類標籤列表
# filenames:文件路徑  ==> 這是使用檔案形式才需要的
# contents:分詞後文件詞向量形式
bunch = Bunch(target_name=[],label=[],contentnames=[],contents=[])


In [20]:
# if os.path.isfile(filepath):
#     jieba.load_userdict(filepath)
# import jieba.posseg as pseg

In [21]:
# create training data - news data
onews1 = "藝人, 八卦，音樂,歌劇,愛不愛"
onews2 = "當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡，不只周亭羽也愛小內，但仍把希望放在正宮小內身上"
onews3 = "中國共產黨第十九次全國代表大會今（18）日揭幕，一連7天在北京人民大會堂舉行，外界關注中共總書記習近平的第二個5任期，可能確立個人集權。對此，前政大國發所所長李酉潭直言，習近平已經無路可走，若不將中共實施民主化會很慘。李酉潭指出，中國大陸政權的領導方式，從鄧小平之後都走集體領導，如今要走回全部權力集中，黨內會有更大反撲；政治學有一個定理，「擁有權力會腐化，擁有絕對的權力會絕對的腐化」，就算習近平不腐化，權力在手中，別人巴不得去巴結，如何避免旁邊的人不腐化？"
onews4 = "於1981年起擔任中國駐聯合國日內瓦辦事處譯員，1987年進入中國外交部國際司"
onews5 = "NBA, 三巨頭55分 雷霆拒絕被逆轉,三分火炮穩住軍心"
onews6 = "NBA,詹姆斯歸隊 騎士仍不敵公牛吞4連敗,每場輸對手10.5分，可見得整個球隊還處在磨合期"

In [22]:
seg_list = jieba.cut(onews3, cut_all=False)
# print "Default Mode:", " ".join(seg_list)  # 預設模式
print " ".join(seg_list)

中國共產黨 第十九次 全國代表大會 今 （ 18 ） 日 揭幕 ， 一連 7 天在 北京人民大會堂 舉行 ， 外界 關注 中共 總書記 習近平 的 第二個 5 任期 ， 可能 確立 個人 集權 。 對此 ， 前 政大 國發 所 所長 李酉潭 直言 ， 習近平 已經 無路可走 ， 若不將 中共 實施 民主化 會 很慘 。 李酉潭 指出 ， 中國 大陸 政權 的 領導 方式 ， 從 鄧小平 之後 都 走 集體領導 ， 如今 要 走 回 全部 權力 集中 ， 黨內 會 有 更 大 反撲 ； 政治學 有 一個 定理 ， 「 擁有 權力 會 腐化 ， 擁有 絕對 的 權力 會 絕對 的 腐化 」 ， 就算 習近平 不 腐化 ， 權力 在 手中 ， 別人 巴不得 去 巴結 ， 如何 避免 旁邊 的 人 不 腐化 ？


In [23]:
news1 = " ".join(jieba.cut(onews1, cut_all=False))
news2 = " ".join(jieba.cut(onews2, cut_all=False))
news3 = " ".join(jieba.cut(onews3, cut_all=False))
news4 = " ".join(jieba.cut(onews4, cut_all=False))
news5 = " ".join(jieba.cut(onews5, cut_all=False))
news6 = " ".join(jieba.cut(onews6, cut_all=False))

In [24]:
labelList = np.array(['a','b','c'])
bunch.target_name.extend(labelList)
# for mylabel in labelList:
bunch.label.append(labelList[0])
bunch.contents.append(news1)
bunch.contentnames.append("news1")
bunch.label.append(labelList[0])
bunch.contents.append(news2)
bunch.contentnames.append("news2")
bunch.label.append(labelList[1])
bunch.contents.append(news4)
bunch.contentnames.append("news4")
bunch.label.append(labelList[1])
bunch.contents.append(news3)
bunch.contentnames.append("news3")
bunch.label.append(labelList[2])
bunch.contents.append(news5)
bunch.contentnames.append("news5")
bunch.label.append(labelList[2])
bunch.contents.append(news6)
bunch.contentnames.append("news6")

#對象持久化                                                                                              
file_obj = open("train_set.dat", "wb")
pickle.dump(bunch,file_obj)                      
file_obj.close()

print "構建文本對象結束！！！"

構建文本對象結束！！！


In [25]:
#引入持久化類
import cPickle as pickle
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer  

# 讀取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

#計算訓練語料的tfidf權值並持久化為詞袋

#讀取bunch對象
def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj) 
    file_obj.close()
    return bunch

#寫入bunch對象
def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj) 
    file_obj.close()

# 構建tf-idf詞向量空間對象
tfidfspace = Bunch(target_name=bunch.target_name,label=bunch.label,contentnames=bunch.contentnames,tdm=[],vocabulary={})
# 使用TfidfVectorizer初始化向量空間模型 
stopword_path = "hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df =1, max_features=30)
# vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df =1,ngram_range=(1,2))
transformer=TfidfTransformer() # 該類會統計每個詞語的tf-idf權值
# 文本轉為詞頻矩陣,單獨保存字典文件 
tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_

# 創建詞袋的持久化
space_path = "tfdifspace.dat"        # 詞向量空間保存路徑
writebunchobj(space_path,tfidfspace)

print "if-idf詞向量空間創建成功！！！"

if-idf詞向量空間創建成功！！！


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(stop_words=stpwrdlst ,decode_error='ignore')
# X_train_counts = count_vect.fit_transform(tfidfspace.vocabulary)
# X_train_counts.shape

tfidfspace.vocabulary
# 三巨頭55分 : \u4e09\u5de8\u982d55\u5206
# 表示斷詞不夠好. 需要建立 vocabulary 以及 token 

{u'nba': 0,
 u'\u4e2d\u5171': 1,
 u'\u4e2d\u570b': 2,
 u'\u64c1\u6709': 3,
 u'\u653f\u6cbb\u5b78': 4,
 u'\u6574\u500b': 5,
 u'\u65b9\u5f0f': 6,
 u'\u65c1\u908a': 7,
 u'\u66dd\u5149': 8,
 u'\u674e\u9149\u6f6d': 9,
 u'\u6b0a\u529b': 10,
 u'\u6b4c\u5287': 11,
 u'\u6b63\u5bae': 12,
 u'\u6b78\u968a': 13,
 u'\u6bcf\u5834': 14,
 u'\u6c11\u4e3b\u5316': 15,
 u'\u706b\u70ae': 16,
 u'\u7121\u8def\u53ef\u8d70': 17,
 u'\u7403\u968a': 18,
 u'\u7576\u5e74': 19,
 u'\u76f4\u8a00': 20,
 u'\u78ba\u7acb': 21,
 u'\u78e8\u5408\u671f': 22,
 u'\u7a69\u4f4f': 23,
 u'\u7b2c\u4e8c\u500b': 24,
 u'\u7b2c\u5341\u4e5d\u6b21': 25,
 u'\u7d55\u5c0d': 26,
 u'\u7e3d\u66f8\u8a18': 27,
 u'\u7fd2\u8fd1\u5e73': 28,
 u'\u8150\u5316': 29}

In [27]:
print tfidfspace.tdm.shape
print tfidfspace.label

(6, 30)
['a', 'a', 'b', 'b', 'c', 'c']


In [28]:
tfidfspace.tdm.data

array([ 1.        ,  0.57735027,  0.57735027,  0.57735027,  1.        ,
        0.13314239,  0.16236593,  0.27490942,  0.16236593,  0.34074314,
        0.16236593,  0.16236593,  0.27490942,  0.16236593,  0.16236593,
        0.16236593,  0.16236593,  0.38745291,  0.16236593,  0.27490942,
        0.38745291,  0.27490942,  0.16236593,  0.50161301,  0.61171251,
        0.61171251,  0.34430007,  0.4198708 ,  0.4198708 ,  0.4198708 ,
        0.4198708 ,  0.4198708 ])

In [38]:
# create test data - news data
tnews1 = "藝人, 八卦，音樂,歌劇,愛不愛, 高什麼"
tnews2 = "劈腿曝光,也愛小內，但仍希望, 當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡"
tnews3 = "一連7天在北京人民大會堂舉行，外界關注中共總書記習近平的第二個5任期，可能確立個人集權。對此，前政大國發所所長李酉潭直言"
tnews4 = "中國外交部國際司,於1981年起擔任中國駐聯合國日內瓦辦事處譯員"
tnews5 = "NBA, 三巨頭55分 雷霆三分"
#tnews6 = "NBA,詹姆斯不敵,磨合期"       
# 上面的測試文字，太短, 會有維度的問題
# tnews1 = "藝人, 八卦，音樂,歌劇,愛不愛"
# tnews2 = "當年九把刀劈腿曝光後，曾在記者會上表明雖然自己2個都很喜歡，不只周亭羽也愛小內，但仍把希望放在正宮小內身上"
# tnews3 = "台灣的政治，國台辦不負責，4名副主任排名依序為"
# tnews4 = "於1981年起擔任中國駐聯合國日內瓦辦事處譯員，1987年進入中國外交部國際司"
# tnews5 = "NBA, 三巨頭55分 雷霆拒絕被逆轉,三分火炮穩住軍心"
# tnews6 = "NBA,詹姆斯歸隊 騎士仍不敵公牛吞4連敗,每場輸對手10.5分，可見得整個球隊還處在磨合期"

In [39]:
tnews1 = " ".join(jieba.cut(tnews1, cut_all=False))
tnews2 = " ".join(jieba.cut(tnews2, cut_all=False))
tnews3 = " ".join(jieba.cut(tnews3, cut_all=False))
tnews4 = " ".join(jieba.cut(tnews4, cut_all=False))
tnews5 = " ".join(jieba.cut(tnews5, cut_all=False))
#tnews6 = " ".join(jieba.cut(tnews6, cut_all=False))

In [40]:
testbunch = Bunch(target_name=[],label=[],contentnames=[],contents=[])
testbunch.target_name.extend(labelList)
# for mylabel in labelList:
testbunch.label.append(labelList[0])
testbunch.contents.append(tnews1)
testbunch.contentnames.append("tnews1")
testbunch.label.append(labelList[2])
testbunch.contents.append(tnews2)
testbunch.contentnames.append("tnews2")
testbunch.label.append(labelList[1])
testbunch.contents.append(tnews4)
testbunch.contentnames.append("tnews4")
testbunch.label.append(labelList[1])
testbunch.contents.append(tnews3)
testbunch.contentnames.append("tnews3")
testbunch.label.append(labelList[2])
testbunch.contents.append(tnews5)
testbunch.contentnames.append("tnews5")
#testbunch.label.append(labelList[2])
#testbunch.contents.append(tnews6)
#testbunch.contentnames.append("tnews6")

#對象持久化                                                                                              
file_obj = open("test_set.dat", "wb")
pickle.dump(testbunch,file_obj)                      
file_obj.close()

print "構建文本測試set結束！！！"

構建文本測試set結束！！！


In [41]:
#test詞向量空間創建
# 構建tf-idf詞向量空間對象
testspace = Bunch(target_name=testbunch.target_name,label=testbunch.label,contentnames=testbunch.contentnames,tdm=[],vocabulary={})
# 使用TfidfVectorizer初始化向量空間模型 
stopword_path = "hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,max_df = 0.5, max_features=30)
# vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,min_df = 1, ngram_range=(1,2))

# 補足不足處
# from sklearn.feature_extraction.text import HashingVectorizer
# hv = HashingVectorizer(n_features=23)


transformer=TfidfTransformer() # 該類會統計每個詞語的tf-idf權值
# 文本轉為詞頻矩陣,單獨保存字典文件 
testspace.tdm = vectorizer.fit_transform(testbunch.contents)
testspace.vocabulary = vectorizer.vocabulary_
# 新增
# testspace.tdm = hv.transform(testbunch.contents)

# 創建詞袋的持久化
space_path = "testspace.dat"        # 詞向量空間保存路徑
writebunchobj(space_path,testspace)

print "test詞向量空間創建成功！！！"

test詞向量空間創建成功！！！


In [42]:
print testspace.label
print testspace.tdm.shape
print testspace.tdm.todense()
print type(testspace.tdm.data)
# testspace.tdm.data = (testspace.tdm.data.flatten())
# testspace.tdm.data.reshape(6,23)
print testspace.tdm.shape

['a', 'c', 'b', 'b', 'c']
(5, 30)
[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.70710678
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.70710678  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.31835597
   0.31835597  0.          0.          0.          0.53902351  0.          0.
   0.31835597  0.          0.          0.          0.          0.          0.
   0.31835597  0.          0.          0.31835597  0.31835597  0.          0.
   0.          0.31835597  0.        ]
 [ 0.5         0.          0.          0.          0.          0.          0.
   0.          0.5         0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.5         0.
   0.          0.          0.         

In [34]:
#預測
#計算分類精度：
def metrics_result(actual,predict):
    print '精度:{0:.3f}'.format(metrics.precision_score(actual,predict,average='macro'))  
    print '召回:{0:0.3f}'.format(metrics.recall_score(actual,predict,average='macro'))  
    print 'f1-score:{0:.3f}'.format(metrics.f1_score(actual,predict,average='macro'))  

In [43]:
# 導入訓練集
trainpath = "tfdifspace.dat"
train_set = readbunchobj(trainpath)

# 導入測試集
testpath = "testspace.dat"
test_set = readbunchobj(testpath)

xlabel = np.array(train_set.label)

print train_set.tdm.shape
print test_set.tdm.shape
print type(train_set.label), train_set.label
print type(xlabel), xlabel.shape
print test_set.contentnames

(6, 30)
(5, 30)
<type 'list'> ['a', 'a', 'b', 'b', 'c', 'c']
<type 'numpy.ndarray'> (6,)
['tnews1', 'tnews2', 'tnews4', 'tnews3', 'tnews5']


In [46]:
from sklearn.metrics import accuracy_score

# 應用樸素貝葉斯算法 
# 1. 輸入詞袋向量和分類標籤
#alpha 平滑參數:0.001 alpha越小，迭代次數越多，精度越高

clf = MultinomialNB(alpha = 0.001).fit(train_set.tdm, train_set.label)

predicted = clf.predict(test_set.tdm)
total = len(predicted);rate = 0
for flabel,contentname,expct_cate in zip(test_set.label,test_set.contentnames,predicted):
    if flabel != expct_cate:
        rate += 1
        print contentname,"實際類別:",flabel," -->預測類別:",expct_cate
# 精度
print "error rate:",float(rate)*100/float(total),"%"
print "預測完畢!!!"

print "準確率：", accuracy_score(test_set.label,predicted)
metrics_result(test_set.label,predicted)

tnews2 實際類別: c  -->預測類別: b
tnews4 實際類別: b  -->預測類別: a
tnews5 實際類別: c  -->預測類別: b
error rate: 60.0 %
預測完畢!!!
準確率： 0.4
精度:0.278
召回:0.500
f1-score:0.356
